In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook #inline
import matplotlib.pyplot as plt

In [ ]:
import remote_control

remote_control.__file__

In [ ]:
from remote_control.control import configure_fly_at_fixed_z
configure_fly_at_fixed_z(2000, 500)

In [ ]:
from remote_control import acquisition
slides = acquisition.SLIDES
#print(slides)
custom_plate =  { 
    "name": "HT Proplate 40 wells",
    "spot_spacing": (4.5, 4.5, 1), #hor,ver (mm) centre to centre distance
    "spot_size": (1 ,1, 1), #h, v (mm)  (wells are (3.5 ,3.5), but acq area is 1x1
    "grid_size": (4,10), # h, v,
    "shape": "rectangle",
}

ga = acquisition.WellPlateGridAquisition(plate_type=custom_plate, 
                                         config_fn = 'remote_config.json', 
                                         datadir="./data", # will save spatial position file here
                                         log_fname="./test.log")

#calibration point locations, corresponds to wells in the next line, and the 'top_left' after 3 lines!
instrument_positions = [
         (422, 4252, -4.4),
         (-10252, 4187, -94.5),
         (29, 42684, 122.5),
         (-10536, 42666, 43)
]
#wells for calibration, series from 0 - (n-1) !!
wells = [
    (0,0,0),
    (3,0,0),   ## is 4th well horizontal, first vertical (counted from bottom)
    (0,9,0),
    (3,9,0),
] 

image_bounds = [
                  [3700, -450],           ## right_bottom
                  [-13900, 48000],        ## left top
                ] # [.max_x, min_y] [min_x, .max_y] 

ga.calibrate(instrument_positions, wells, 'top_left') #calibration point location!
ga.set_image_bounds(image_bounds)

#plt.figure()
#plt.scatter([c[0] for c in instrument_positions], [c[1] for c in instrument_positions])
#plt.show()

#plt.figure()
#plt.scatter([c[0] for c in instrument_positions], [c[1] for c in wells])
#plt.show()

spotsToUse = [
(0, 0),
(1, 0),
(2, 0),
(3, 0),
(0, 1),
(1, 1),
(2, 1),
(3, 1),
(0, 2),
(1, 2),
(2, 2),
(3, 2),
(0, 3),
(1, 3),
(2, 3),
(3, 3),
(0, 4),
(1, 4),
(2, 4),
(3, 4),
(0, 5),
(1, 5),
(2, 5),
(3, 5),
(0, 6),
(1, 6),
(2, 6),
(3, 6),
(0, 7),
(1, 7),
(2, 7),
(3, 7),
(0, 8),
(1, 8),
(2, 8),
(3, 8),
(0, 9),
(1, 9),
(2, 9),
(3, 9)
    
]

# Generate coordinates & preview
ga.generate_targets(
    spotsToUse, 
    offset_x=200,    ### 200 um (!) offset to prevent acquisition at the extreme top
    offset_y=200, 
    pixelsize_x=10, 
    pixelsize_y=10, 
    area_function_name='upper_left')   ## upper_left, as this is also the calibration point of earlier steps

ga.plot_targets(annotate=True)

# Write coordinates to file locally

In [ ]:
ga.write_imzc_coords_file('./data/positions_imzc.txt')
ga.write_json_coords_file('./data/positions.json')

# Perform acquisition & write to file in output directory

In [ ]:
filename = "./data/xxxx" # Prefix for output files

ga.acquire(filename, dummy=True)